# Instalando bibliotecas

In [ ]:
""" Installing libraries """
%pip install pandas==2.3.2 matplotlib==3.10.6 seaborn==0.13.2 scikit-learn==1.7.1 numpy==2.2.6 pyarrow==21.0.0 torch==2.8.0

# Importando bibliotecas (externas e próprias)

In [1]:
""" Importing libraries """

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Our modules
from Modules.loading.read_parquet import read_parquet_file 
from Modules.preprocessing.onehot import one_hot_encode_parquet
from Modules.models.NBeats import NBeatsBlock, NBeats
from Modules.models.training import train_model_NBeats

# Definição dos hiper-parâmetros

In [ ]:
""" Defining hyper-parameters """
# Neural Network Global Parameters
input_size = 7  # Number of past days to use as input
output_size = 1  # Number of future days to predict
batch_size = 30  # Batch size for training

n_layers = 4  # Number of layers in the N-BEATS model
hidden_size = 128  # Number of hidden units in each layer

# Training parameters
learning_rate = 0.001  # Learning rate for the optimizer
epochs = 50  # Number of training epochs (iterations over the entire dataset)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

# Importação do Dataset

É interessante dividir o treino em batches (mini-conjuntos de treino). Cada batch possui o tamanho de input size, seguindo a ordem cronológica de vendas dentro daquela janela de dias. No entanto, durante o treinamento é **ESSENCIAL** que a escolha do próximo batch seja aleatória.

Ex.: Inicia o treino por 21-27 jul e prevê 28, depois pula para 02-08 fev para prever 03. Esse processo deve ser repetido até todos os dados serem treinados, finalizando **01 epoch**.

O número de **epochs** diz o número total de iterações do modelo com relação ao dataset inteiro.

Sobre a composição da janela de input dentro de um batch, existem duas abordagens:

1) Treinar em cada janela todas as séries (pense que cada par produto-loja x tempo representa uma série temporal dentro daquele período). Esse modelo é bem mais complexo pois o output deve ter o mesmo tamanho de produto-loja.
2) Treinar vários modelos separados (considerando uma série temporal para cada modelo). Esse método é ineficiente pois o modelo nunca irá aprender os padrões entre as séries.
3) Treinar o modelo com um par produto-loja por vez. Ou seja:
   - O modelo realiza epochs = N iterações de treino ao longo de todo dataset
     - Em cada epoch, passa por todas as M batches
       - Em cada batch (que possui uma janela de tamanho input_size), atualiza os parâmetros para cada série temporal ($x_l,y_l$). Totalizando L atualizações, com L sendo o número de pares produto-loja.

Ressalta-se que cada conjunto ($x_l,y_l$) representa:
- $x_l$: série temporal do l-ésimo par produto-loja, sendo um vetor de tamanho input_size x (features + 1)
- $y_l$: Previsão de vendas do l-ésimo par produto-loja para os próximos $output_size$ dias


In [7]:
""" Setting up the constants """

DATA_PATH = "../Data/hackathon_2025_templates/part-00000-tid-"

FILE_HASH = [
    "2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36",
    "5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8",
    "7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1",
]

SUFIX = "-4-1-c000.snappy.parquet"

FILE_NAMES = [DATA_PATH + hash + SUFIX for hash in FILE_HASH]

In [8]:
""" Loading the data """

pdv_data = read_parquet_file(FILE_NAMES[0])
transaction_data = read_parquet_file(FILE_NAMES[1])
product_data = read_parquet_file(FILE_NAMES[2])

In [9]:
pdv_data.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [10]:
transaction_data.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


In [11]:
product_data.head()

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc


# Reorganizando os dados em pares produto-loja

In [ ]:

# 1) juntar produto
df = transaction_data.merge(
    product_data,
    left_on="internal_product_id",
    right_on="produto",
    how="left"
)

# 2) juntar loja
df = df.merge(
    pdv_data,
    left_on="internal_store_id",
    right_on="pdv",
    how="left"
)

# 3) garantir que a data esteja no formato datetime
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# 4) pivotar: dia x par produto-loja
full_df = (
    df.pivot_table(
        index="transaction_date",                        # agora usa transaction_date
        columns=["internal_product_id", "internal_store_id"],  
        values="quantity",                               # mantenha 'quantity' se esse for o nome certo
        aggfunc="sum",
        fill_value=0
    )
)

# 5) deixar colunas mais legíveis: produto_loja
full_df.columns = [f"{p}_{l}" for p, l in full_df.columns]

# 6) garantir todas as datas do período (mesmo sem vendas)
full_df = full_df.reindex(
    pd.date_range(df["transaction_date"].min(), df["transaction_date"].max(), freq="D"),
    fill_value=0
)
full_df.index.name = "data"


            1000423277513436210_1164003957453133052  \
data                                                  
2022-01-01                                      0.0   
2022-01-02                                      0.0   
2022-01-03                                      0.0   
2022-01-04                                      0.0   
2022-01-05                                      0.0   

            1000423277513436210_1870860275432322673  \
data                                                  
2022-01-01                                      0.0   
2022-01-02                                      0.0   
2022-01-03                                      0.0   
2022-01-04                                      0.0   
2022-01-05                                      0.0   

            1000423277513436210_2096630570628676364  \
data                                                  
2022-01-01                                      0.0   
2022-01-02                                      0.0   
2022-01-

# Transformando em um Dataset

In [24]:
class SalesDataset(Dataset):
    def __init__(self, tabela, input_size, output_size):
        self.X = []
        self.y = []

        # Para cada coluna (cada par produto-loja)
        for col in tabela.columns:
            series = tabela[col].values
            n_samples = len(series) - input_size - output_size + 1

            for i in range(n_samples):
                x_window = series[i:i+input_size]
                y_window = series[i+input_size:i+input_size+output_size]

                self.X.append(x_window)
                self.y.append(y_window)

        # Converte para tensores PyTorch
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

        # Se quiser adicionar dimensão de features (ex.: 1 venda + n_features)
        self.X = self.X.unsqueeze(-1)   # shape = [n_samples_totais, input_size, 1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ================================
# Cria dataset e dataloader
# ================================
dataset = SalesDataset(tabela, input_size, output_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

KeyboardInterrupt: 